In [360]:
import numpy as np
import sympy as sp
import copy

In [456]:
SIZE = 4
presicion = 10

In [457]:
def get_indexes_max_nondiag(Matrix):
    indexes = (1, 0)
    for i in range(Matrix.shape[0]):
        for j in range(i):
            indexes = (i, j) if abs(Matrix[indexes]) < abs(Matrix[i, j]) else indexes
    return indexes


def get_max_diag(Matrix):
    max_value = 0
    for i in range(Matrix.shape[0]):
        max_value = abs(Matrix[i, i]) if max_value < abs(Matrix[i, i]) else max_value
    return max_value


def sgn(value):
    return 1 if value >= 0 else -1


def validation_defination(Matrix, sigma_elem):
    obstacle = np.sqrt(get_max_diag(Matrix)) * sigma_elem
    for i in range(Matrix.shape[0]):
        for j in range(Matrix.shape[1]):
            if (i!=j and abs(Matrix[i, j]) > obstacle):
                return True
    return False

    
def rotations_with_obstacles(Matrix, p):
    sigmas = [10**(-k) for k in range(p)]
    W = sp.eye(Matrix.shape[0])
    for sigma_elem in sigmas:
        while (validation_defination(Matrix, sigma_elem)):
            row_max, col_max = get_indexes_max_nondiag(Matrix)
            
            d = np.sqrt((Matrix[row_max, row_max] - Matrix[col_max, col_max])**2 + 4*(Matrix[row_max, col_max]**2))
            c = np.sqrt((1+(abs(Matrix[row_max, row_max] - Matrix[col_max, col_max]) / d)) / 2)
            s = sgn(Matrix[row_max, col_max] * (Matrix[row_max, row_max] - Matrix[col_max, col_max])) * np.sqrt((1-(abs(Matrix[row_max, row_max] - Matrix[col_max, col_max]) / d)) / 2)
            
            C_matrix = copy.deepcopy(Matrix)
            
            for k in range(Matrix.shape[0]):
                for l in range(Matrix.shape[1]):
                    if k != row_max and k != col_max and l != row_max and l != col_max:
                        None
                    elif k != row_max and k != col_max:
                        C_matrix[k, row_max] = C_matrix[row_max, k] = c * Matrix[k, row_max] + s * Matrix[k, col_max]
                        C_matrix[k, col_max] = C_matrix[col_max, k] =  - s * Matrix[k, row_max] + c * Matrix[k, col_max]
            C_matrix[row_max, row_max] = c**2 * Matrix[row_max, row_max] + c*s*2*Matrix[row_max, col_max] + s**2 * Matrix[col_max, col_max]
            C_matrix[col_max, col_max] = s**2 * Matrix[row_max, row_max] - c*s*2*Matrix[row_max, col_max] + c**2 * Matrix[col_max, col_max]
            C_matrix[row_max, col_max] = C_matrix[col_max, row_max] = 0
            
            T = sp.eye(Matrix.shape[0])
            T[row_max, row_max] = c
            T[row_max, col_max] = -s
            T[col_max, row_max] = s
            T[col_max, col_max] = c
            
            W *= T
            
            Matrix = C_matrix
    
    lambdas = [Matrix[i, i] for i in range(Matrix.shape[0])]
    
    return Matrix, lambdas, np.matrix([[round(W[i, j], 8) for j in range(SIZE)] for i in range(SIZE)])

In [458]:
A = np.matrix([
    [-0.168700, 0.353699, 0.008540, 0.733624],
    [0.353699, 0.056519, -0.723182, -0.076440],
    [0.008540, -0.723182, 0.015938, 0.342333],
    [0.733624, -0.076440, 0.342333, -0.045744]
], dtype=np.float64)

SIZE = A.shape[0]

A

matrix([[-0.1687  ,  0.353699,  0.00854 ,  0.733624],
        [ 0.353699,  0.056519, -0.723182, -0.07644 ],
        [ 0.00854 , -0.723182,  0.015938,  0.342333],
        [ 0.733624, -0.07644 ,  0.342333, -0.045744]])

In [459]:
Result_Matrix, eigenvalues, vectors = rotations_with_obstacles(A, presicion)

sp.Matrix(Result_Matrix)

Matrix([
[  -0.943567855487324,  9.70642139548909e-13, 2.56187264309855e-21,                     0],
[9.70642139548909e-13,     0.857774177202395,                    0, -2.56187232160398e-21],
[2.56187264309855e-21,                     0,   -0.744036406029614,  9.70642131905946e-13],
[                   0, -2.56187232160398e-21, 9.70642131905946e-13,     0.687843084314543]])

In [460]:
(np.array(eigenvalues), vectors)

(array([-0.94356786,  0.85777418, -0.74403641,  0.68784308]),
 matrix([[0.72360715, -0.02236707, 0.04102068, 0.68862887],
         [-0.33173569, 0.64347358, 0.60387191, 0.33351447],
         [-0.04102065, -0.68862871, 0.72360729, -0.02236710],
         [-0.60387204, -0.33351456, -0.33173559, 0.64347346]], dtype=object))

In [461]:
np.linalg.eig(A)

(array([-0.94356786, -0.74403641,  0.68784308,  0.85777418]),
 matrix([[ 0.72360715, -0.04102068,  0.68862887,  0.02236707],
         [-0.33173569, -0.60387191,  0.33351447, -0.64347358],
         [-0.04102065, -0.72360729, -0.0223671 ,  0.68862871],
         [-0.60387204,  0.33173559,  0.64347346,  0.33351457]]))

In [462]:
print("Результат:", np.sort(eigenvalues))

Результат: [-0.94356786 -0.74403641  0.68784308  0.85777418]


In [463]:
print("Погрешность:", np.sort(eigenvalues) - np.sort(np.linalg.eig(A)[0]))

Погрешность: [ 2.22044605e-16  5.55111512e-16 -3.33066907e-16  1.11022302e-16]
